In [1]:
import pandas as pd
import stanza 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_excel("data/sentiment.xlsx")

In [3]:
# Load the English model
nlp = stanza.Pipeline('en')
# Perform frequency analysis
word_frequency = {}

2023-09-10 12:14:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-09-10 12:17:02 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-09-10 12:17:02 INFO: Using device: cpu
2023-09-10 12:17:02 INFO: Loading: tokenize
2023-09-10 12:17:02 INFO: Loading: pos
2023-09-10 12:17:02 INFO: Loading: lemma
2023-09-10 12:17:02 INFO: Loading: constituency
2023-09-10 12:17:02 INFO: Loading: depparse
2023-09-10 12:17:02 INFO: Loading: sentiment
2023-09-10 12:17:03 INFO: Loading: ner
2023-09-10 12:17:03 INFO: Done loading processors!


In [15]:
import re

# Function to remove URLs using regular expression
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

def frequency_analysis(text):
    # Process the text
    doc = nlp(text)

    # Define words and characters to exclude
    exclude_words = {'anz', 'anzbank'}  # Add any other words to exclude
    exclude_characters = {',', '.', '#'}

    for sentence in doc.sentences:
        for word in sentence.words:
            word_text = word.text.lower()  # Convert to lowercase
            # Remove URLs from the word text
            word_text = remove_urls(word_text)
            
            # Check if the word should be excluded
            if word_text not in exclude_words and word_text not in exclude_characters:
                if word_text not in word_frequency:
                    word_frequency[word_text] = 1
                else:
                    word_frequency[word_text] += 1

In [16]:
# Apply frequency analysis to the "raw_content" column and store results in "word_frequency"
df['raw_content'].apply(frequency_analysis)

0       None
1       None
2       None
3       None
4       None
        ... 
6340    None
6341    None
6342    None
6343    None
6344    None
Name: raw_content, Length: 6345, dtype: object

In [17]:
# Sort words by frequency (most frequent first)
sorted_word_frequency = dict(sorted(word_frequency.items(), key=lambda item: item[1], reverse=True))

# Print the most frequent words and their frequencies
# num_top_keywords = 10  # Number of top keywords to display
# top_keywords = list(sorted_word_frequency.keys())[:num_top_keywords]
keyworddf=[]
keywords = list(sorted_word_frequency.keys())
for keyword in keywords:
    keyworddf.append([keyword, sorted_word_frequency[keyword]])
    # print(f"Keyword: {keyword}, Frequency: {sorted_word_frequency[keyword]}")

In [19]:
index_labels = ['Keyword', 'Frequency']

pd.DataFrame(keyworddf, columns=index_labels).to_excel("data/frequency_analysis.xlsx", index=False)